In [1]:
import sys

setupFlag = False ## Set flag to True if these aren't installed
if setupFlag:
    !{sys.executable} -m pip install astroquery
    ## https://github.com/astropy/astroquery
    !{sys.executable} -m pip install eleanor
    ## https://github.com/afeinstein20/eleanor
    !{sys.executable} -m pip install lightkurve
    ## https://github.com/KeplerGO/lightkurve

In [2]:
import csv
import dis
import inspect
import os
import sys

import astropy
import astroquery
import eleanor
#import tess_stars2px ## Currently unnecessary
import lightkurve as lk
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import scipy as sp
import time
import warnings
warnings.filterwarnings('ignore')

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.time import TimeDelta
from astropy.timeseries import TimeSeries
from astropy.visualization import time_support
time_support()
from astropy.visualization import quantity_support
quantity_support()

from scipy import linalg as la
from scipy import optimize
from scipy import integrate
from scipy import stats

from IPython.display import display_html
from IPython.display import Image

In [3]:
def mkdir(directory): ## creates a directory if it doesn't exist
    ## credit to https://gist.github.com/keithweaver/562d3caa8650eefe7f84fa074e9ca949
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

def display_side_by_side(*args): ##displays pandas DataFrames side by side
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

def unravel(list): ## creates an array from a list of arrays
    return np.array([i for array in list for i in array])

savePNG = True ## Changes matplotlib backend to save plots as pgf (default:True)
if savePNG:
    mpl.use("agg")
    plotExt = str('.png')
elif not savePNG:
    mpl.use("pgf")
    mpl.rcParams.update({
        "pgf.texsystem": "pdflatex",
        'font.family': 'serif',
        'text.usetex': True,
        'pgf.rcfonts': False,})
    plotExt = str('.pgf')

notebookPlotFlag = True ## Changes Jupyter plotting backend (default:True)
if notebookPlotFlag:
    %matplotlib notebook
elif not notebookPlotFlag:
    %matplotlib inline

In [4]:
def interpToMatch(item1,item2):
    ## Function takes two pandas DataFrames with a 'time' column (float or integer type)
    ## and interpolates the data to match the set with a smaller number of points
    ## with the interpolated DataFrames being returned as a tuple
    item1_indexed = item1.set_index('time')
    item2_indexed = item2.set_index('time')
    #display(item1_indexed)
    item1_length = len(item1_indexed.index)
    item2_length = len(item2_indexed.index)
    #display(item1_length)
    if item1_length >= item2_length:
        minun = item2_indexed.index.min()
        plusle = item2_indexed.index.max()
        numPoints = item2_length
    elif item1_length <= item2_length:
        minun = item1_indexed.index.min()
        plusle = item1_indexed.index.max()
        numPoints = item1_length
    #display(minun)
    #display(plusle)
    
    #numPoints = abs(plusle-minun)
    newIndex = np.linspace(minun,plusle-1,numPoints)
    #display(numPoints)
    #display(newIndex)
    
    item1_interp = pd.DataFrame(index=newIndex)
    item1_interp.index.name = item1_indexed.index.name
    item2_interp = pd.DataFrame(index=newIndex)
    item2_interp.index.name = item2_indexed.index.name

    for colname, col in item1_indexed.iteritems():
        item1_interp[colname] = np.interp(newIndex,item1_indexed.index,col)
    for colname, col in item2_indexed.iteritems():
        item2_interp[colname] = np.interp(newIndex,item2_indexed.index,col)
    item1_interp.reset_index(inplace=True)
    item2_interp.reset_index(inplace=True)
    
    return item1_interp, item2_interp

In [5]:
def interpToData(data, *args):
    ## More generalized version of interpToMatch(). Takes an argument for a reference
    ## DataFrame and a variable number of DataFrames to be interpolated so that
    ## they match the time sampling of the reference DataFrame. Like interpToMatch(),
    ## DataFrames must have a 'time' column of an integer or float type.
    ## Function returns an array containing the reference DataFrame as the first
    ## item followed by the interpolated DataFrames in the order in which they were
    ## passed to the function
    interpArray = []
    interpArray.append(data)
    
    data_indexed = data.set_index('time')
    data_length = len(data_indexed.index)
    minun = data_indexed.index.min()
    plusle = data_indexed.index.max()
    newIndex = data_indexed.index
    
    for arg in args:
        arg_indexed = arg.set_index('time')
        arg_interp = pd.DataFrame(index=newIndex)
        arg_interp.index.name = arg_indexed.index.name
        for colname, col in arg_indexed.iteritems():
            arg_interp[colname] = np.interp(newIndex,arg_indexed.index,col)
        arg_interp.reset_index(inplace=True)
        interpArray.append(arg_interp)
    return interpArray

In [6]:
def snStats(radec, peak, sector=None, tpfSize=23, apRad=1,
            plot=True, savePlot=False, targetName=None, verbose=False):
    ## Uses eleanor to search for target SN in TESS field and identifies if peak is observed
    ## during a given sector. Function returns both the converted number of days
    ## relative to the estimated peak as well as the lightkurve object produced
    ## for the source.
    
    ## radec is taken as string in form 'hh mm ss +dd mm ss'
    ## peak is taken as string in form 'yyyy-mm-dd hh:mm:ss' or 'yyyy-mm-dd' (ie iso format)
    ## sector is taken as integer between 1-23, signifying which sector is of interest 
    ##     Note: if sector argument is not called, function will identify the most
    ##     recent sector when the target was viewed. Not all objects are viewed in all 
    ##     sectors, Eleanor documentation explains how one can determine what sectors an 
    ##     object is viewed in.
    ## tpfSize is taken as an integer and is the size of produced target pixel file in pixels 
    ##     Note: must be an odd number, defaults to 23 pixels as that seems to be the size used
    ##     in Vallely et al.
    ## plot is taken as a boolean and plots the tpf and the flux relative to the estimated peak
    ## savePlot is taken as a boolean and saves the plots to a local folder
    ## targetName is taken as a string and alters plot titles and save directory
    ##     Note: if targetName is not provided, plots will be labeled as the identified TIC
    ##     of the object and, if savePlot is True, the directory for saved plots will be
    ##     named according to the TIC as well.
    ## verbose is taken as a boolean and will return various statistics about the observation
    
    ## Note: should try to make it so there's an argument that allows for the auto-aperture to be used
    ## Note: should remove dependence on lightkurve and swap to using pandas DataFrames
    
    coords = SkyCoord(radec,unit=(u.hourangle,u.deg));
    if sector:
        target = eleanor.Source(coords=coords,sector=sector);
    elif not sector:
        target = eleanor.Source(coords=coords);
    data = eleanor.TargetData(target, height=tpfSize, width=tpfSize, do_psf=False, do_pca=False);
    
#     eleanor.TargetData.custom_aperture(data, shape='circle', r=apRad, method='exact')
#     eleanor.TargetData.get_lightcurve(data)
    
    ticCoords = SkyCoord(ra=target.coords[0],dec=target.coords[1],unit=(u.deg,u.deg))
    #time.sleep(0.15) ## originally in place due to file caching issues, re-enable if these are encountered
    peakTime = Time(peak, format='iso')
    obsStartTime = lk.utils.btjd_to_astropy_time(data.time)[0]
    obsEndTime = lk.utils.btjd_to_astropy_time(data.time)[-1]
    peakTimePres = obsStartTime.jd <= peakTime.jd <= obsEndTime.jd
    if not peakTimePres:
        timeDiff = np.around(-peakTime.to_value('jd') + 
                             min([obsStartTime.to_value('jd'), obsEndTime.to_value('jd')], 
                                 key=lambda x:abs(x-peakTime.to_value('jd'))),2)
        print("Warning: provided peak is "+str(timeDiff)+" days outside of TESS time range")
    q = data.quality == 0    
    lc = data.to_lightkurve(flux=data.raw_flux+0.1*data.flux_bkg)
    daysFromPeak = lk.utils.btjd_to_astropy_time(lc.time) - peakTime.jd 
    obsPeak = Time(np.str(lc.time[np.argmax(lc.flux)]+2457000),format='jd')
    
    if verbose:
        print()
        print("Target Name: "+targetName if targetName else "")
        print("TESS Input Catalog (TIC) ID: "+str(target.tic))
        print("Located at: "+ticCoords.to_string('hmsdms'), end=" ")
        print("(exact target)" if ticCoords.ra==coords.ra and 
              ticCoords.dec==coords.dec else "(closest target)")
        print("TESS Sector: "+str(target.sector))
        print("Observation Start: "+obsStartTime.iso)
        print("Observation End: "+obsEndTime.iso)
        print("Observation Length: "+
              str(np.around(obsEndTime.to_value('jd') - obsStartTime.to_value('jd'),3))+' days')
        print("Peak Flux of "+str(np.around(np.max(lc.flux),3))+
              " e/s Observed on "+str(obsPeak.iso))
        print()
    if plot:
        fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(15,4))
        ax1.imshow(data.tpf[0])
        ax1.set_title('Target Pixel File')
        ax2.imshow(data.bkg_tpf[0])
        ax2.set_title('2D interpolated background')
        ax3.imshow(data.aperture)
        ax3.set_title('Aperture')
        plotTitle = str(targetName+' (TIC '+
                        str(target.tic)+')') if targetName else str("Identified Target (TIC "
                                                                        +str(target.tic)+")")
        fig.suptitle(plotTitle)
        if savePlot:
            saveDir = str("./"+targetName+"/") if targetName else str("./TIC "+str(target.tic)+"/")
            mkdir(saveDir)
            fig.savefig(saveDir+"TPF"+plotExt, bbox_inches='tight')
        plt.show()
        
        plt.figure(figsize=(15,5))
        plt.plot(daysFromPeak, lc.flux, color=(0,0.6,0.5), label="Lightcurve")
        ## originally plotted the maximum value of the flux, disabled
        ## because it is currently dominated by points at the start of
        ## TESS' orbit. May be brought back after that data is excluded
        #plt.axvline(x=daysFromPeak.jd[np.argmax(lc.flux)], color=(0,0,0),
                   #linestyle='dotted', label='Max Flux: '+str(np.around(np.max(lc.flux),3))+' e/s')
        if peakTimePres:
            plt.axvline(x=0, color=(0.8,0.4,0), 
                        linestyle='dashed', label='Estimated Peak')
        plt.locator_params(axis='x', nbins=10)
        plt.xticks(rotation=20)
        plt.xlabel('Days From Estimated Peak')
        plt.ylabel('Flux(e/s)')
        plt.legend()
        plt.title(plotTitle)
        if savePlot:
            saveDir = str("./"+targetName+"/") if targetName else str("./TIC "+str(target.tic)+"/")
            mkdir(saveDir)
            fig.savefig(saveDir+"lightcurve"+plotExt, bbox_inches='tight')
        plt.show()
        
    return [daysFromPeak, lc]

In [7]:
def optBackground(t, initGuess=0.2, plot=True):
    ## Function finds the optimal percentage of background flux to add back in to match results of Vallely et al.
    ## for ASASSN-18tb by using a chi squared penalty function
    ## t is taken as a tuple, where the two terms are the min and max day values over which to optimize the penalty
    ## function
    ## initGuess is the first guess for the percent of background flux to add back in (should be around 0.00-0.50)
    ## Note: would possibly be good to generalize this function in the event that data from other papers on 
    ## other supernovae can be found
    
    paperCurve = pd.read_csv('asassn18tb_TESS_Sector1.txt',delim_whitespace=True,header=0,names=["time","counts","error"])
    paperCurve['counts_median'] = paperCurve['counts'].rolling(12).median()
    paperCurve['error_median'] = paperCurve['error'].rolling(12).median()
    t0 = t[0]
    t1 = t[1]

    source = eleanor.Source(coords=SkyCoord('04 18 06.149 −63 36 56.68',
                                    unit=(u.hourangle,u.deg)),sector=1)
    data = eleanor.TargetData(source,height=17,width=17,
                         do_psf=False,do_pca=True)
    eleanorCurve = pd.DataFrame()
    q = data.quality == 0
    eleanorCurve["time"] = data.time[q]
    eleanorCurve["raw_flux"] = data.raw_flux[q]
    eleanorCurve['raw_counts'] = data.raw_flux[q] * 1800
    eleanorCurve['raw_counts_median'] = eleanorCurve['raw_counts'].rolling(12).median()
    eleanorCurve["bkg_flux"] = data.flux_bkg[q]
    eleanorCurve['bkg_counts'] = data.flux_bkg[q] * 1800
    eleanorCurve["raw_flux_err"] = data.flux_err[q]
    eleanorCurve['zp_flux'] = eleanorCurve['raw_flux'] - eleanorCurve['raw_flux'].median()
    eleanorCurve['zp_bkg_flux'] = eleanorCurve['bkg_flux'] - eleanorCurve['bkg_flux'].median()
    eleanorCurve['zp_counts'] = eleanorCurve['zp_flux'] * 1800
    eleanorCurve['zp_counts_median'] = eleanorCurve['zp_counts'].rolling(12).median()
    eleanorCurve['zp_bkg_counts'] = eleanorCurve['zp_bkg_flux'] * 1800

    interp_paperCurve = interpToData(eleanorCurve,paperCurve)[1]
    trim_eleanorCurve = eleanorCurve.loc[(eleanorCurve["time"] >= t0) & (eleanorCurve["time"] <= t1)]
    trim_paperCurve = interp_paperCurve.loc[(interp_paperCurve['time'] >= t0) & (interp_paperCurve['time'] <= t1)]
    #display(trim_paperCurve)
    chiSq = lambda x:  np.sum((((trim_eleanorCurve['zp_counts'] + x*trim_eleanorCurve['zp_bkg_counts'])
                      - (trim_paperCurve['counts']))/trim_paperCurve['error'] )**2)
    bkgScale = optimize.fmin(chiSq,initGuess)[0]
    print()
    eleanorCurve['corr_flux'] = eleanorCurve['zp_flux'] + bkgScale * eleanorCurve['zp_bkg_flux']
    eleanorCurve['corr_counts'] = eleanorCurve['corr_flux'] * 1800
    eleanorCurve['corr_counts_median'] = eleanorCurve['corr_counts'].rolling(12).median()
    
    if plot:
        ax = paperCurve.plot(x='time',y='counts',color='blue',alpha=0.1,kind='scatter')
        paperCurve.plot(x='time',y='counts_median',color='blue',linewidth=2,label='Vallely et al.',ax=ax)
        eleanorCurve.plot(x='time',y='zp_counts',color='red',alpha=0.1,kind='scatter',ax=ax)
        eleanorCurve.plot(x='time',y='zp_counts_median',color='red',linewidth=2,label='Raw TESS Observations',ax=ax)
        eleanorCurve.plot(x='time',y='corr_counts',color='green',alpha=0.1,kind='scatter',ax=ax)
        eleanorCurve.plot(x='time',y='corr_counts_median',color='green',linewidth=2,label='Corrected TESS Observations',ax=ax)
        plt.axvline(x=t0,color='black',linestyle='--',label="Sampled Region")
        plt.axvline(x=t1,color='black',linestyle='--')
        #plt.tight_layout()
        plt.ylabel('counts')
        plt.legend()
        plt.title('Optimisation of eleanor Background Subtraction \n for ASASSN-18tb (scale: '+str(bkgScale)+')')
    return eleanorCurve